# Init Libraries

In [ ]:
import pandas as pd
import streamlit as st
import pickle


# Load Data, Preprocess Movies

In [ ]:
movies = pd.read_csv("csv/movies.csv")
links = pd.read_csv("csv/links.csv")
ratings = pd.read_csv("csv/ratings.csv")
movies = movies.merge(links,on="movieId",how="left")
movies["year"] = movies["title"].str[-5:-1]
movies = movies.drop(["imdbId"],axis=1).copy()
movies = movies.drop_duplicates().copy()
movies = movies.loc[movies["title"] != ""]
movies = movies.sort_values(["title","year"]).copy()

# Aggregate Ratings by movieId

In [ ]:
ratings.groupby(["movieId"], as_index=False).agg({"rating":"sum"}).describe()

In [ ]:
aggRatings = ratings.groupby('movieId').agg(Rating_Count=('rating', 'count'),
                                            Rating_Mean=('rating', 'mean')).reset_index()
aggRatings = aggRatings.sort_values("Rating_Count",ascending=False)

# Merge Ratings to Movies

In [ ]:
movies = movies.merge(aggRatings,on="movieId",how="left").copy()

In [ ]:
movies

# Pickle Movielist

In [ ]:
movies = movies.loc[~movies["tmdbId"].isna()].copy()
movies = movies.loc[~movies["Rating_Count"].isna()].copy()

In [ ]:
movies["tmdbId"] = movies["tmdbId"].astype(int)
movies["Rating_Count"] = movies["Rating_Count"].astype(int)

In [ ]:
movies = movies.loc[movies["Rating_Count"]>=5].sort_values("Rating_Mean",ascending=False)

In [ ]:
fileObj = open('pickle/movielist.sav', 'wb')
pickle.dump(movies[["movieId","title","genres","year","tmdbId","Rating_Count","Rating_Mean"]],fileObj)
fileObj.close()

# Build Genre-List

In [ ]:
genres = movies['genres'].str.split('|')
genres = genres.explode('genres')
genrelist = genres.unique()

# and Pickle it as a list

In [ ]:
fileObj = open('pickle/genres.sav', 'wb')
pickle.dump(genrelist,fileObj)
fileObj.close()

# Check pickles

In [ ]:
movielist = pickle.load(open('pickle/movielist.sav', 'rb'))
movielist

In [ ]:
genres = pickle.load(open('pickle/genres.sav', 'rb'))
genres

# some checks and trials

In [115]:
udml = movies
udml["tmdbId"][0:5]

8938      3009
8731    359940
5589     11423
8247     37916
9659     25538
Name: tmdbId, dtype: int32

In [117]:
udml["tmdbId"][0:1]

8938    3009
Name: tmdbId, dtype: int32